In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/06 15:13:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/06 15:13:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/06 15:13:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
# read the Dataset

Training=spark.read.csv('test3.csv',header=True,inferSchema=True)

In [5]:
Training.show()

+----+---+----------+------+
|Name|Age|Experience|Salary|
+----+---+----------+------+
| nik| 28|         5| 15000|
|dany| 45|        10| 10000|
| piu| 25|         3| 25000|
| raj| 22|         4| 12000|
| sam| 19|         2| 15200|
|tiku| 26|         5| 53000|
|sanu| 29|         6| 45000|
+----+---+----------+------+



In [7]:
Training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [8]:
Training.columns

['Name', 'Age', 'Experience', 'Salary']

In [9]:
###[Age,Experience]---->new feature----->independent feature

In [12]:
from pyspark.ml.feature import VectorAssembler
featureAssembler=VectorAssembler(inputCols=['Age','Experience'],outputCol='Independent Feature')

In [13]:
output=featureAssembler.transform(Training)

In [15]:
output.show()

+----+---+----------+------+-------------------+
|Name|Age|Experience|Salary|Independent Feature|
+----+---+----------+------+-------------------+
| nik| 28|         5| 15000|         [28.0,5.0]|
|dany| 45|        10| 10000|        [45.0,10.0]|
| piu| 25|         3| 25000|         [25.0,3.0]|
| raj| 22|         4| 12000|         [22.0,4.0]|
| sam| 19|         2| 15200|         [19.0,2.0]|
|tiku| 26|         5| 53000|         [26.0,5.0]|
|sanu| 29|         6| 45000|         [29.0,6.0]|
+----+---+----------+------+-------------------+



In [16]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Feature']

In [17]:
finalized_data=output.select('Independent Feature','Salary')

In [18]:
finalized_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|         [28.0,5.0]| 15000|
|        [45.0,10.0]| 10000|
|         [25.0,3.0]| 25000|
|         [22.0,4.0]| 12000|
|         [19.0,2.0]| 15200|
|         [26.0,5.0]| 53000|
|         [29.0,6.0]| 45000|
+-------------------+------+



In [19]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Feature', labelCol='Salary')
regressor=regressor.fit(train_data)

23/01/06 15:27:53 WARN Instrumentation: [90ce7739] regParam is zero, which might cause numerical instability and overfitting.


In [20]:
# Coefficeint
regressor.coefficients

DenseVector([-2857.1843, 8613.701])

In [21]:
# Intercept
regressor.intercept

62680.26124818611

In [22]:
## Prediction
pred_result=regressor.evaluate(test_data)

In [23]:
pred_result.predictions.show()

+-------------------+------+------------------+
|Independent Feature|Salary|        prediction|
+-------------------+------+------------------+
|         [28.0,5.0]| 15000|25747.605224963707|
+-------------------+------+------------------+



In [24]:
pred_result.meanAbsoluteError,pred_result.meanSquaredError

(10747.605224963707, 115511018.07166716)